In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pygplates
import pygmt

import collections

import sys
sys.path.append('../python')
import joyful_geochemistry as joy
import joyful_mapping as joymap
#import time_series as ts

%matplotlib inline
%load_ext autoreload
%autoreload 2


In [ ]:
model_dir = '../luffi/REM_surfaces_csv/'
gc_interpolator_dict = joy.make_gc_interpolator_dict(model_dir)

#df = pd.read_csv('../datafiles/geochem_merge_20221026.csv')
df = pd.read_csv('../datafiles/geochem_merge_20231117.csv')
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs=4326)
df = joy.compute_ratios(df, long_list=True)

df = joymap.select_orogens(df,gdf=None, 
               orogen_names='Cordilleran', 
               continent_names='South America',
               region=[-100, -50, -60, 20])



In [ ]:
region = [-85, -55, -58, 12]

def get_residuals(elevation_estimates, df_filt):
    DEM = pygmt.grdtrack(points=pd.DataFrame(data={'x':df_filt.geometry.x,'y':df_filt.geometry.y}),
                         grid=topo, 
                         newcolname='z', 
                         interpolation='l',
                         no_skip=True)['z']
    return elevation_estimates.sub(DEM, axis=0)#.drop(columns=['bin_latitude', 'bin_longitude', 'geometry'])

topo = pygmt.datasets.load_earth_relief(region='d', resolution='02m', registration='gridline')
topo - pygmt.grdcut(topo, region=region)
topo = pygmt.grdfilter(topo, distance=1, spacing='0.2d', filter='m100')


In [ ]:
region = [-85, -55, -58, 12]
#region = [-76, -66, -25, -12]
projection="M10c"
perspective = [240, 35]
column_marker_size='0.3c'

space_bin_size = 1.

age_max = 1.0



#plot_calibrations = collections.OrderedDict()
plot_calibrations = [
    ('luffi', 41, 'LD22 all'),
    ('luffi', 'la_yb_elevation', 'LD22 La/Yb'),
    ('Hu', 'la_yb_elevation', 'H20 La/Yb'),
    ('Hu', 'sr_y_elevation', 'H20 Sr/Y'),
    ('FarnerLee', 'la_yb_elevation', 'FL17 La/Yb'),
    ('FarnerLee', 'gd_yb_elevation', 'FL17 Gd/Yb')
]

#plot_calibrations

#mohometer_selection


for calibration, mohometer_selection, label_string in plot_calibrations:
    
    if isinstance(mohometer_selection, list):
        mohometer_description_string = '|'.join(mohometer_selection)
    else:
        mohometer_description_string = str(mohometer_selection)
    
    df_filt = joy.filter_the_database(df, filter_method=calibration, age_min=-1, age_max=age_max)
    elevation_estimates = joy.get_elevations(df_filt, calibration=calibration, 
                                             mohometer_selection=mohometer_selection,
                                             gc_interpolator_dict=gc_interpolator_dict)
    
    DEM = pygmt.grdtrack(points=pd.DataFrame(data={'x':df_filt.geometry.x,'y':df_filt.geometry.y}),
                         grid=topo, 
                         newcolname='z', 
                         no_skip=True)['z']
    elevations_residuals = elevation_estimates.sub(DEM, axis=0)
    
    binned_elevations = joy.bin_elevations(df_filt.geometry, elevation_estimates, space_bin_size)
    binned_residuals = joy.bin_elevations(df_filt.geometry, elevations_residuals, space_bin_size)
    
    joymap.residuals_crossplot(binned_elevations, binned_residuals,
                               fname='../images/crossplot_{:s}_{:s}_PresentDay.png'.format(calibration,
                                                                                           mohometer_description_string))
    
    # RESIDUALS
    fig = pygmt.Figure()
    joymap.plot_elevation_basemap(fig, topo, cmap='cubhelix',
                           region=region, projection=projection, perspective=perspective)
    pygmt.makecpt(cmap='polar', series=[-3000,3000,500], background='o')
    residual_marker_size = '{:f}{:s}'.format(float(column_marker_size[:-1])*1.5, column_marker_size[-1])
    fig.plot(x=binned_residuals.x, y=binned_residuals.y, 
             fill=binned_residuals.median_elevation, # np.array(elevations_residuals.z)-np.array(elevations_residuals.median_elevation), 
             style='s{:s}'.format(residual_marker_size), pen='0.2,black', perspective=perspective, cmap=True)
    add_colorbar = False
    if (calibration=='FarnerLee') and (mohometer_selection=='gd_yb_elevation'):
        add_colorbar = True
    joymap.add_labels(fig, reconstruction_time=None, label_string=label_string, x=2.3, add_colorbar=add_colorbar, colorbar_title='Residual Elevation [m]')

    fig.savefig('../images/elevation_residuals_{:s}_{:s}_PresentDay.png'.format(calibration,
                                                                                mohometer_description_string))
    
    
    # ELEVATIONS
    fig = pygmt.Figure()
    joymap.plot_elevation_basemap(fig, topo, cmap='cubhelix',
                               region=region, projection=projection, perspective=perspective)
    joymap.plot_elevations_as_columns(fig, binned_elevations, cmap='cubhelix',
                                       column_marker_size='0.2c', column_pen='0.2p,black',
                                       column_delta_height=500, 
                                       region=region, perspective=perspective)
    joymap.add_labels(fig, reconstruction_time=None, add_colorbar=True)
    fig.savefig('../images/elevations_plus_volcanism_{:s}_{:s}_PresentDay.png'.format(calibration,
                                                                                      mohometer_description_string))

#elevation_estimates


In [ ]:
# USE THE BINS INSTEAD OF RAW ESTIMATES...

#stds = df_plot.std()
plot_calibrations = ['luffi', 'Hu', 'FarnerLee']

residual_dict = collections.OrderedDict()

for calibration in plot_calibrations:
    
    df_filt = joy.filter_the_database(df, filter_method=calibration, age_min=-1, age_max=age_max)
    elevation_estimates = joy.get_elevations(df_filt, calibration=calibration,
                                             gc_interpolator_dict=gc_interpolator_dict)
    
    residual_dict[calibration] = get_residuals(elevation_estimates, df_filt)



fig = plt.figure(figsize=(12,5))
gs = fig.add_gridspec(nrows=1, ncols=3, wspace=0.2, hspace=.7, width_ratios=[41,2,2])

for i,calibration in enumerate(residual_dict.keys()): #[residuals_luffi, residuals_Hu, residuals_FarnerLee]):
    print(calibration)
    ax=fig.add_subplot(gs[i])
    rs = [v[~np.isnan(v)] for v in residual_dict[calibration].to_numpy().T]
    ax.boxplot(rs, widths=0.8,
               showfliers=False, 
               patch_artist=True,
               labels=residual_dict[calibration].columns,
               medianprops={"linewidth": 5,
                 "solid_capstyle": "butt",
                 'color': 'red'},
               boxprops={'facecolor':'red',
                         'alpha':0.3,
                         'edgecolor':'black'},)
    ax.set_ylim(-5000,5000)
    ax.grid(axis='y')
    ax.set_xticklabels([s[:-10].replace('feo_tot','feo(tot)').replace('_ppm',' [ppm]').replace('_','/') for s in residual_dict[calibration].columns], rotation = 75)
    if i>0:
        ax.set_yticklabels([])
    else:
        ax.set_ylabel('Residual Elevation (Estimate - DEM) [m]')

plt.savefig('../images/comparison_by_mohometer.png')
plt.show()
